In [24]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import csv
import os

In [25]:
url = 'https://www.imdb.com/list/ls062911411/?sort=user_rating,desc&st_dt=&mode=detail&page=1'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
# response    --> <Response [200]> means it is successful
# print(soup) --> prints the html of the entire page.

In [26]:
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
link = []
descriptions = []
genres = []

In [27]:
# When you hover over the movie and click inspect, you can see that the movie information is stored in this div class.
movie_data = soup.findAll('div', attrs = {'class' : 'lister-item mode-detail'}) 
# movie_data --> I can see movie details inside div class

In [28]:

txt_folder = os.path.join("txt")
csv_folder = os.path.join("csv")

all_movies_txt_filename = os.path.join(txt_folder, "all_movies.txt")
with open(all_movies_txt_filename, mode="w", encoding='utf-8-sig') as all_movies_file:
    for index, point in enumerate(movie_data, start=1):
        
        txt_filename = os.path.join(txt_folder, f"movie{index}.txt")
        with open(txt_filename, mode="w", encoding='utf-8-sig') as file:
            name = point.h3.a.text
            movie_name.append(name)

            new_link = 'https://www.imdb.com' + point.h3.a.get('href')
            link.append(new_link)
            new_url = requests.get(new_link)
            new_soup = BeautifulSoup(new_url.content, 'html.parser')

            release_year = point.h3.find('span', class_='lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
            year.append(release_year)

            run_time = point.p.find('span', class_='runtime').text.replace(' min', '')
            time.append(run_time)

            rate = point.find('div', class_='ipl-rating-star small').text.replace('\n', '')
            rating.append(rate)

            meta = point.find('span', class_='metascore').text.replace(' ', '') if point.find('span', class_='metascore') else '^^^^^^'
            metascore.append(meta)

            value = point.find_all('span', attrs={'name': 'nv'})
            vote = value[0].text.replace(' ', '')
            votes.append(vote)

            grosses = value[1].text if len(value) > 1 else '*****'
            gross.append(grosses)

            file.write(f"Name: {name}\n")
            file.write(f"Release Year: {release_year}\n")
            file.write(f"Run Time: {run_time}\n")
            file.write(f"Rating: {rate}\n")
            file.write(f"Metascore: {meta}\n")
            file.write(f"Votes: {vote}\n")
            file.write(f"Gross: {grosses}\n")
            file.write(f"Link: {new_link}\n")

            all_movies_file.write(f"Movie {index}:\n")
            all_movies_file.write(f"Name: {name}\n")
            all_movies_file.write(f"Release Year: {release_year}\n")
            all_movies_file.write(f"Run Time: {run_time}\n")
            all_movies_file.write(f"Rating: {rate}\n")
            all_movies_file.write(f"Metascore: {meta}\n")
            all_movies_file.write(f"Votes: {vote}\n")
            all_movies_file.write(f"Gross: {grosses}\n")
            all_movies_file.write(f"Link: {new_link}\n")
            all_movies_file.write("\n") 

    csv_filename = os.path.join(csv_folder, "movies_data.csv")
    with open(csv_filename, mode="w", newline='', encoding='utf-8-sig') as csv_file:
        writer = csv.writer(csv_file)
        header = ['Name of movie', 'Year of relase', 'Watchtime', 'Movie rating', 'Metascore', 'Votes', 'Gross', 'Link']
        writer.writerow(header)

        for i in range(len(movie_name)):
            writer.writerow([movie_name[i], year[i], time[i], rating[i], metascore[i], votes[i], gross[i], link[i]])

In [29]:
# NumPy library to find the number of non-zero elements in a NumPy array
# np.count_nonzero(movie_name) --> 100

In [30]:
movie_DF = pd.DataFrame({'Name of movie' : movie_name, 'Year of relase' : release_year, 'Watchtime' : time, 'Movie rating' : rating, 'Metascore' : metascore, 'Votes' : votes, 'Gross' : gross, 'Link': link})

In [31]:
movie_DF


,Name of movie,Year of relase,Watchtime,Movie rating,Metascore,Votes,Gross,Link
0,The Shawshank Redemption,1960,142,9.3,82,"2,848,909",$28.34M,https://www.imdb.com/title/tt0111161/
1,The Godfather,1960,175,9.2,100,"1,985,100",$134.97M,https://www.imdb.com/title/tt0068646/
2,The Dark Knight,1960,152,9,84,"2,830,280",$534.86M,https://www.imdb.com/title/tt0468569/
3,Schindler's List,1960,195,9,95,"1,431,164",$96.90M,https://www.imdb.com/title/tt0108052/
4,The Lord of the Rings: The Return of the King,1960,201,9,94,"1,951,788",$377.85M,https://www.imdb.com/title/tt0167260/
...,...,...,...,...,...,...,...,...
95,Vertigo,1960,128,8.3,100,"422,481",$3.20M,https://www.imdb.com/title/tt0052357/
96,Singin' in the Rain,1960,103,8.3,99,"257,545",$8.82M,https://www.imdb.com/title/tt0045152/
97,Toy Story 3,1960,103,8.3,92,"882,881",$415.00M,https://www.imdb.com/title/tt0435761/
98,To Kill a Mockingbird,1960,129,8.3,88,"329,929",*****,https://www.imdb.com/title/tt0056592/
